# Setup Data Mesh - Ingestão e Criação de Tabelas

Este notebook é responsável por:
1. Inicializar a Sessão Spark.
2. Criar os Bancos de Dados Lógicos (Schemas) para cada domínio.
3. Ler os arquivos CSV brutos.
4. Salvar as tabelas no formato gerenciado (Parquet/Delta) com `overwrite`.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pyspark.sql import SparkSession

# Garante que o diretório app seja visível
sys.path.append(os.getcwd())

In [ ]:
# Inicialização da Sessão Spark
# Se estiver rodando localmente, isso cria um warehouse local.
# Se estiver no Databricks, usa a sessão ativa.
spark = SparkSession.builder \
    .appName("SetupDataMesh") \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark Session Ativa")

In [ ]:
# Configuração dos Caminhos
DATA_PATH = "data/"

# Definição dos Domínios e Arquivos
DOMAINS = {
    "olist_sales": [
        "olist_orders_dataset.csv", 
        "olist_order_items_dataset.csv", 
        "olist_products_dataset.csv"
    ],
    "olist_logistics": [
        "olist_sellers_dataset.csv", 
        "olist_geolocation_dataset.csv", 
        "olist_customers_dataset.csv"
    ],
    "olist_finance": [
        "olist_order_payments_dataset.csv"
    ],
    "olist_cx": [
        "olist_order_reviews_dataset.csv"
    ]
}

In [ ]:
def ingest_domain(domain_name, file_list):
    print(f"\n>>> Configurando Domínio: {domain_name}")
    
    # 1. Criar Schema/Database
    print(f"Creating Schema {domain_name}...")
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {domain_name}")
    
    # 2. Iterar arquivos e criar tabelas
    for filename in file_list:
        file_path = os.path.join(DATA_PATH, filename)
        
        if not os.path.exists(file_path):
            print(f"  [SKIP] Arquivo não encontrado: {file_path}")
            continue
            
        # Nome da tabela limpo
        table_name = filename.replace("olist_", "").replace("_dataset.csv", "")
        full_table_name = f"{domain_name}.{table_name}"
        
        print(f"  Ingestão: {filename} -> {full_table_name}")
        
        # Leitura do CSV
        try:
            df = spark.read.option("header", "true") \
                           .option("inferSchema", "true") \
                           .csv(file_path)
            
            # Escrita com Overwrite (SaveAsTable)
            # Isso cria a tabela no Metastore e move os dados para o Warehouse
            df.write.mode("overwrite").saveAsTable(full_table_name)
            print(f"  [OK] Tabela {full_table_name} atualizada com sucesso.")
            
        except Exception as e:
            print(f"  [ERRO] Falha ao processar {filename}: {str(e)}")

In [ ]:
# Execução Geral
for domain, files in DOMAINS.items():
    ingest_domain(domain, files)
    
print("\n=== Data Mesh Setup Concluído ===")

In [ ]:
# Validação: Listar tabelas criadas
for domain in DOMAINS.keys():
    print(f"\nTabelas em {domain}:")
    spark.sql(f"SHOW TABLES IN {domain}").show(truncate=False)